In [0]:
import pandas as pd

In [0]:
S1 = pd.read_csv('/Volumes/Gibbs_Drive/Sentinel_Test/Sen1_Data_Full.csv', parse_dates = ['DATE'])
S2 = pd.read_csv('/Volumes/Gibbs_Drive/Sentinel_Test/Sen2_Dates.csv', parse_dates = ['DATE']) 
FW = pd.read_csv('/Volumes/Gibbs_Drive/Sentinel_Test/FW_Dates.csv', parse_dates = ['DATE'])

In [0]:
#Get only GRD products
S1 = S1[(S1['TYPE'] == 'GRD')]

#Reverse FW Dataframe
FW = FW.sort_index(axis=1 ,ascending=False)

In [0]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [0]:
#Find nearest scene <= to 7 days from field date for both S1 & S2
#If scene name is the same as the last scene name, use the next closest scene
csv_data = list()

for i, item in enumerate(list(FW['DATE'])):
    #For every field date - find the date, name, days elapsed of nearest S1 & S2 scene
    S1_Near = nearest(list(S1['DATE']), item)
    S1_Name = S1[S1['DATE'] == S1_Near]['NAME']
    S1_Num = (S1_Near - item).days
    
    S2_Near = nearest(list(S2['DATE']), item)
    S2_Name = S2[S2['DATE'] == S2_Near]['TITLE']
    S2_Num = (S2_Near - item).days
    
    #Check that the name of this file is not the same as the preceding
    if i >= 1:
      if str(S1_Name) == str(csv_data[i - 1][3]):
        print('S1 Name matches for ' + str(csv_data[i-1][0]))
        #If the filename is the same, find the next nearest scene and record its name
        S1_Temp = S1
        S1_Temp = S1_Temp.drop(S1[S1['DATE'] == S1_Near].index)
        #These will overwrite previous Near, Name, Num variables
        S1_Near = nearest(list(S1_Temp['DATE']), item)
        S1_Name = S1_Temp[S1_Temp['DATE'] == S1_Near]['NAME']
        S1_Num = (S1_Near - item).days
    
    if i >= 1:
      if str(S2_Name) == str(csv_data[i - 1][6]):
        print('S2 Name matches for ' + str(csv_data[i-1][0]))
        #If the filename is the same, find the next nearest scene and record its name
        S2_Temp = S2
        S2_Temp = S2_Temp.drop(S2[S2['DATE'] == S2_Near].index)
        #These will overwrite previous Near, Name, Num variables
        S2_Near = nearest(list(S2_Temp['DATE']), item)
        S2_Name = S2_Temp[S2_Temp['DATE'] == S2_Near]['TITLE']
        S2_Num = (S2_Near - item).days
    
    
    #Save these scenes in a list
    csv_data.append((item, S1_Near, S1_Num, S1_Name, S2_Near, S2_Num, S2_Name))
    
    
    #S1_Near = nearest(list(S1['DATE']), item)
    #S1_Num = (S1_Near - item).days
    #S2_Near = nearest(list(S2['DATE']), item)
    #S2_Num = (S2_Near - item).days
    #S2_Title = S2[S2['DATE'] == S2_Near].iloc[0,3]
    #csv_data.append((item, S1_Near, S1_Num, S2_Near, S2_Num, S2_Title))


In [0]:
Nearest_DF = pd.DataFrame(csv_data)
Nearest_DF.columns = ['FW', 'S1', 'S1_NUM', 'S1_NAME', 'S2', 'S2_NUM', 'S2_NAME']
Nearest_DF


In [0]:
#This doesn't give any more than 4 scenes either, stick with Scene_Date_Match_1 and downloaded files
Final_Scenes = Nearest_DF[(Nearest_DF["S1_NUM"] <= 7) & (Nearest_DF["S1_NUM"] >= -7) & (Nearest_DF["S2_NUM"] <= 7) & (Nearest_DF["S2_NUM"] >= -7)]
Final_Scenes

In [0]:
Nearest_DF.to_csv('/Volumes/Gibbs_Drive/Sentinel_Test/Nearest_Dates.csv')
Final_Scenes.to_csv('/Volumes/Gibbs_Drive/Sentinel_Test/Final_Scenes.csv')